In [111]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [112]:
train_data = pd.read_csv("../input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [113]:
test_data = pd.read_csv("../input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [114]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier

In [115]:
X_train = train_data
y_train = train_data["Survived"]
y_test = test_data
X_test = test_data
X_train.drop(['Survived'], axis=1, inplace=True)
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

numeric_features = ["PassengerId", "Pclass", "Age", "SibSp", "Parch", "Fare"]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))])

categorical_features = ["Sex", "Embarked"]
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
        ])

my_pipe = Pipeline(steps=[('preprocessor', preprocessor)])
X_transformed_train = my_pipe.fit_transform(X_train, y_train)
X_transformed_test = my_pipe.transform(X_test)
X_transformed_train, X_transformed_valid, y_train, y_valid = train_test_split(X_transformed_train, y_train, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=1)
model.fit(X_transformed_train, y_train)
prediction = model.predict(X_transformed_valid)

In [118]:
print('MAE:', mean_absolute_error(y_valid, prediction))
#model.fit(X_transformed_test, y_test)
#X_transformed_train_OH = my_pipe.transform(X_transformed_test).toarray()
print(X_transformed_test.shape)
print(X_transformed_test)
print(X_transformed_test[22])

MAE: 0.22346368715083798
(418, 12)
[[8.920e+02 3.000e+00 3.450e+01 ... 1.000e+00 0.000e+00 0.000e+00]
 [8.930e+02 3.000e+00 4.700e+01 ... 0.000e+00 1.000e+00 0.000e+00]
 [8.940e+02 2.000e+00 6.200e+01 ... 1.000e+00 0.000e+00 0.000e+00]
 ...
 [1.307e+03 3.000e+00 3.850e+01 ... 0.000e+00 1.000e+00 0.000e+00]
 [1.308e+03 3.000e+00 2.800e+01 ... 0.000e+00 1.000e+00 0.000e+00]
 [1.309e+03 3.000e+00 2.800e+01 ... 0.000e+00 0.000e+00 0.000e+00]]
[914.       1.      28.       0.       0.      31.6833   1.       0.
   0.       0.       1.       0.    ]


In [119]:
prediction_test = model.predict(X_transformed_test)

In [122]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': prediction_test})
output.to_csv('my_submission_v2.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
